In [1]:
import cv2
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dropout, BatchNormalization, GaussianNoise, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
size = 512
channels =  1  #input image format
input_img = Input(shape=(size, size, channels))
#print(input_img)

In [4]:
# Encoder
x = GaussianNoise(0.1)(input_img)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Dropout(0.2)(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Dropout(0.2)(x)

# Decoder
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Dropout(0.2)(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Dropout(0.2)(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')



In [5]:
def preprocess_image(image):
    # Resize the image to ensure consistent input size
    image = cv2.resize(image, (size, size))

    # Apply Gaussian Blurring to smooth the image and reduce noise
    image = cv2.GaussianBlur(image, (5, 5), 0)

    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Convert the image to 8-bit if it's not already
    if image.dtype != np.uint8:
        image = np.uint8(image)
    
    # Apply adaptive thresholding to create a binary image
    # This will help in highlighting significant features
    image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                  cv2.THRESH_BINARY, 11, 2)
    
    # Use morphological closing to close small holes in the foreground
    kernel = np.ones((5, 5), np.uint8)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)

    # Dilate the result to make the objects more apparent
    image = cv2.dilate(image, kernel, iterations=1)

    # Edge detection to highlight boundaries
    image = cv2.Canny(image, 100, 200)

    # Normalize the image to have values between 0 and 1
    image = np.expand_dims(image, axis=-1) / 255.0
    
    return image

def custom_preprocessor(img):
    # Assuming 'img' is a NumPy array of image data
    return preprocess_image(img)

# Setup the data generator
datagen = ImageDataGenerator(
    preprocessing_function=custom_preprocessor,
    validation_split=0.15  # if you have validation data
)


In [6]:
train_generator = datagen.flow_from_directory(
    'dataset',  #path to dataset
    target_size=(size, size),
    color_mode='grayscale',
    batch_size=16,
    class_mode='input',
    subset='training'
)
validation_generator = datagen.flow_from_directory(
    'dataset',  #path to dataset
    target_size=(size, size),
    color_mode='grayscale',
    batch_size=16,
    class_mode='input',
    subset='validation'
)

Found 152 images belonging to 6 classes.
Found 25 images belonging to 6 classes.


In [7]:
# Define early stopping callback
early_stopper = EarlyStopping(monitor='val_loss', patience=30, verbose=1, restore_best_weights=True)

# Now include the callback in the fit method
autoencoder.fit(
    train_generator,
    epochs=200,
    validation_data=validation_generator,
    callbacks=[early_stopper]
)

Epoch 1/200


/Users/yong/anaconda3/envs/plard/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 31s 3s/step - loss: 0.1798 - val_loss: 0.2402
Epoch 2/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 30s 3s/step - loss: 0.1164 - val_loss: 0.2284
Epoch 3/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 30s 3s/step - loss: 0.0718 - val_loss: 0.2166
Epoch 4/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 31s 3s/step - loss: 0.0421 - val_loss: 0.2058
Epoch 5/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - loss: 0.0266 - val_loss: 0.1965
Epoch 6/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 58s 6s/step - loss: 0.0177 - val_loss: 0.1894
Epoch 7/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 48s 5s/step - loss: 0.0131 - val_loss: 0.1837
Epoch 8/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 40s 4s/step - loss: 0.0106 - val_loss: 0.1788
Epoch 9/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 44s 4s/step - loss: 0.0091 - val_loss: 0.1745
Epoch 10/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 38s 4s/step - loss: 0.0082 - val_loss: 0.1708
Epoch 11/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 40s 4s/step - loss: 0.0076 - val_loss: 0.1675
Epoch 12/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 39s 4s/step - loss: 0.0071 - val_lo

KeyboardInterrupt: 

In [8]:
#save model
autoencoder.save('0507test.keras')